In [28]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import gc
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
import random 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
regr = LinearRegression()

In [29]:
sum1 = 0
sum2 = 0
sum3 = 0
N = 1
for i in range(N):
    df_train = pd.read_csv('c://train2.csv',encoding='GB2312')
    # df_train

    for c, dtype in zip(df_train.columns, df_train.dtypes):
        if dtype == np.float64:
            df_train[c] = df_train[c].astype(np.float32)

    df_train = df_train[df_train['血糖'] < 20]
#     df_train = df_train[df_train['血糖'] > 3]
    print(i)

    df_train.fillna(df_train.median(),inplace = True)
    # df_train.fillna(0,inplace = True)
    df_train = df_train.replace('男',1)
    df_train = df_train.replace('女',0)
    df_train = df_train.replace('??',0)

    x_train  = df_train.drop(['血糖','体检日期','id'], axis=1) #,'白球比例' ,'*球蛋白'  ,,'乙肝e抗原','id'
#     print(x_train)
    # x_train = x_train.drop(['乙肝表面抗原', '乙肝表面抗体', '乙肝e抗原', '乙肝e抗体', '乙肝核心抗体'], axis=1)
    # x_train

    y_train = df_train['血糖'].values

    y_train

    x_train, test_feature, y_train, test_labels = train_test_split(
            x_train, y_train, test_size=0.0001, random_state=i +1 )
    # x_train, test_feature, y_train, test_labels = train_test_split(
    #         x_train, y_train, test_size=0.1, random_state=1)
    y_train
    y_pred = regr.fit(x_train, y_train).predict(test_feature)
    y_pred

    train_columns = x_train.columns

    for c in x_train.dtypes[x_train.dtypes == object].index.values:
        x_train[c] = (x_train[c] == True)

    del df_train; gc.collect()


    x_train = x_train.values.astype(np.float32, copy=False)
    d_train = lgb.Dataset(x_train, label=y_train)

    params = {}
    params['max_bin'] = 10
    params['learning_rate'] =0.0165# shrinkage_rate
    params['boosting_type'] = 'gbdt'
    params['objective'] = 'regression'
    params['metric'] = 'mae'          # or 'mae'
    params['sub_feature'] = 0.58     # feature_fraction -- OK, back to .5, but maybe later increase this
    params['bagging_fraction'] = 0.7 # sub_row
    params['bagging_freq'] = 2
    params['num_leaves'] = 12        # num_leaf
    params['min_data'] = 10        # min_data_in_leaf
    params['min_hessian'] = 0.01     # min_sum_hessian_in_leaf
    params['verbose'] = 0

    clf = lgb.train(params, d_train, 450)

    del d_train; gc.collect()
    del x_train; gc.collect()

    # df_test  = pd.read_csv('c://test.csv')
    df_test = test_feature

    x_test = df_test[train_columns]
    x_test = x_test.replace('男',1)
    x_test = x_test.replace('女',0)
    # x_test

    for c in x_test.dtypes[x_test.dtypes == object].index.values:
        x_test[c] = (x_test[c] == True)

    x_test = x_test.values.astype(np.float32, copy=False)


    clf.reset_parameter({"num_threads":1})
    p_test = clf.predict(x_test)
    len(p_test)

    

    len(test_labels)


    for i in range(len(p_test)):
        sum1 += (test_labels[i] - p_test[i]) ** 2
        sum3 += (test_labels[i] - y_pred[i]) ** 2
        sum2 += (test_labels[i] - pd.Series(y_train).mean()) ** 2

    #     sum2 += (test_labels[i] - 7) ** 2
print(sum1 / len(test_labels)/N/2   )
print(sum3 / len(test_labels)/N/2  )
print(sum2 / len(test_labels)/N/2   )
print()


0
0.872895314226
0.969906531166
1.96700239936



In [30]:
df_test  = pd.read_csv('c://test.csv')
# df_test = test_feature
df_test.fillna(df_test.median(),inplace = True)
df_test  = df_test.drop(['体检日期','id'], axis=1)

In [31]:
# df_test 

In [32]:
x_test = df_test[train_columns]
x_test = x_test.replace('男',1)
x_test = x_test.replace('女',0)
# x_test

In [33]:
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)

In [34]:
x_test = x_test.values.astype(np.float32, copy=False)


In [35]:
clf.reset_parameter({"num_threads":1})
p_test = clf.predict(x_test)
len(p_test)


1000

In [36]:
pd.Series(p_test).to_csv('output_lgbt2.csv', index=False)